In [ ]:
google_colab = True
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)
    base_path = '/content/drive/MyDrive/MasterThesis/'
else:
    base_path = '../.'

Mounted at /content/drive


In [ ]:
if google_colab:
    !pip install socceraction matplotsoccer
    !pip install pandas
    !pip install tables == 3.5.1
    !pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 197 kB 24.9 MB/s 
  Created wheel for matplotsoccer: filename=matplotsoccer-0.0.8-py3-none-any.whl size=5983 sha256=82f0bf4e8654973da24e47dbf3752dfbec057113603c6fe5d0def955d72c5219
  Stored in directory: /root/.cache/pip/wheels/be/6f/64/335f5a5e7763b284f2f7d8893f20338de4d9f73794fe85ea7d
Successfully built matplotsoccer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Invalid requirement: '=='
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 137 kB/s 


In [ ]:

import os
import json
import warnings
import shutil
import json
import pandas as pd
from matplotlib import pyplot as plt
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from enum import Enum

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
import socceraction.vaep.formula as vaepformula
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import nn
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss
from sklearn.metrics import roc_curve, confusion_matrix, precision_recall_curve

import plotly.graph_objects as go
import plotly.figure_factory as ff

if google_colab:
    % load_ext autoreload
    % autoreload 2
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))


class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

1
Tesla P100-PCIE-16GB


In [ ]:
if google_colab:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = "cpu"
seed = 42


def seed_new_generator():
    return torch.Generator().manual_seed(seed)


os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True
    print(torch.cuda.current_device())

0


In [ ]:
if google_colab:
    os.system(f'git clone https://github.com/statsbomb/open-data {base_path}')

In [ ]:
if google_colab:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

    from psutil import virtual_memory

    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Mon Aug  1 18:54:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class StatsBombDataset(Enum):
    BARCELONA = 1
    WOMEN = 2

In [ ]:
dataset_version = StatsBombDataset.BARCELONA

if dataset_version == StatsBombDataset.BARCELONA:
    datafolder = os.path.join(base_path, "data-barcelona/")
elif dataset_version == StatsBombDataset.WOMEN:
    datafolder = os.path.join(base_path, "data-women/")

archive_path = "./"

spadl_h5 = os.path.join(datafolder, "spadl-statsbomb.h5")
print(os.path.exists(spadl_h5))
features_h5 = os.path.join(datafolder, "features.h5")
labels_h5 = os.path.join(datafolder, "labels.h5")
predictions_h5 = os.path.join(datafolder, "predictions.h5")
vaep_values_h5 = os.path.join(datafolder, "vaep_values.h5")

neural_features_h5 = os.path.join(datafolder, "neural_features.h5")
neural_predictions_h5 = os.path.join(datafolder, "neural_predictions.h5")
substitutions_h5 = os.path.join(datafolder, "substitutions.h5")

player_features_h5 = os.path.join(datafolder, "player_features.h5")
player_substitutions_h5 = os.path.join(datafolder, "player_substitutions.h5")
player_list_per_season_json = os.path.join(datafolder, "player_list_per_season.json")
player_list_name_per_season_json = os.path.join(datafolder, "player_list_name_per_season.json")
cumsum_current_player_stats_npy = os.path.join(datafolder, "cumsum_current_player_stats.npy")
current_player_stats_npy = os.path.join(datafolder, "cumulated_player_stats.npy")
cumsum_history_player_stats_npy = os.path.join(datafolder, "cumsum_history_player_stats.npy")
history_player_stats_npy = os.path.join(datafolder, "history_player_stats.npy")
player_lineups_npy = os.path.join(datafolder, "player_lineups")
game_substitutions_npy = os.path.join(datafolder, "game_substitutions")
home_player_lineups_npy = os.path.join(datafolder, "home_player_lineups")
away_player_lineups_npy = os.path.join(datafolder, "away_player_lineups")
home_game_substitutions_npy = os.path.join(datafolder, "home_game_substitutions")
away_game_substitutions_npy = os.path.join(datafolder, "away_game_substitutions")

SBL = StatsBombLoader(root=os.path.join(base_path, 'data'), getter='local')

assert os.path.exists(datafolder)
print(home_game_substitutions_npy)
# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)


True
/content/drive/MyDrive/MasterThesis/data-barcelona/home_game_substitutions


In [ ]:
class ActionDataset(Dataset):
    def __init__(self, neural_features_h5, Xcols, Ycols, time_column_name, seq_size=1):
        self.games = pd.read_hdf(neural_features_h5, "game_list")

        self.game_indices_list = {}
        self.game_features_list = {}
        self.game_times_list = {}
        self.game_target_list = {}
        self.neural_features = []
        self.time = []
        self.target = []
        self.action_indices = []
        self.Xcols = Xcols
        self.Ycols = Ycols
        self.length = 0
        self.seq_size = seq_size

        for game in tqdm(list(self.games.itertuples()), desc='Loading data'):
            game_neural_features = pd.read_hdf(neural_features_h5, f'neural_features/game_{game.game_id}')
            game_index = game_neural_features.index
            game_features = game_neural_features[Xcols].to_numpy()
            game_times = game_neural_features[[time_column]].to_numpy()
            game_target = game_neural_features[Ycols].to_numpy()

            game_action_indices = game_index[:len(game_neural_features) - (seq_size - 1)]
            game_action_game_id = np.full((len(game_action_indices)), game.game_id)
            game_indices_and_id = np.stack((game_action_indices, game_action_game_id), axis=1)

            self.action_indices.append(game_indices_and_id)

            self.game_indices_list[game.game_id] = np.array(
                [game_index[i:i + seq_size] for i in range(len(game_index) - (seq_size - 1))])
            self.game_features_list[game.game_id] = torch.as_tensor(game_features, device=device)
            self.game_times_list[game.game_id] = torch.as_tensor(game_times, device=device)
            self.game_target_list[game.game_id] = torch.as_tensor(game_target, device=device)

        self.action_indices = np.concatenate(self.action_indices, axis=0)
        self.over_sampled = False
        self.over_sampled_target = None

    def __len__(self):
        return self.action_indices.shape[0]

    def __getitem__(self, item):
        if self.over_sampled:
            item = self.over_sampled_target.iloc[item].indices
        index_to_extract, game_id = self.action_indices[item][0], self.action_indices[item][1]
        index_sequence = self.game_indices_list[game_id][index_to_extract]
        event_sequence = self.game_features_list[game_id][index_sequence]
        time_sequence = self.game_times_list[game_id][index_sequence]
        target_sequence = self.game_target_list[game_id][index_to_extract + self.seq_size - 1]
        return item, game_id, index_sequence, torch.as_tensor(event_sequence), torch.as_tensor(
            time_sequence).float(), torch.as_tensor(target_sequence).float()

    def get_target(self):
        return self.target

    def apply_over_sampling(self):
        self.over_sampled = True
        X = self.target
        X = pd.DataFrame(X, columns=['target'])
        X['indices'] = X.index
        num_0 = len(X[X['target'] == 0])
        oversampled_data = pd.concat(
            [X[X['target'] == 0], X[X['target'] == 1].sample(num_0, replace=True, random_state=42)])
        self.over_sampled_target = oversampled_data.sample(frac=1).reset_index(drop=True)
        self.length = len(self.over_sampled_target)
        print(f'Oversampling applied, new dataset lenght : {self.length}')

In [ ]:
from sklearn.preprocessing import LabelEncoder

features_real_columns = ['start_x_a0',
                        'start_y_a0',
                        'end_x_a0',
                        'end_y_a0',
                        'dx_a0',
                        'dy_a0',
                        'movement_a0',
                        'start_dist_to_goal_a0',
                        'start_angle_to_goal_a0',
                        'end_dist_to_goal_a0',
                        'end_angle_to_goal_a0',
                        ]
features_discrete_columns = [
    'goalscore_diff',
    'goalscore_team',
    'goalscore_opponent',
    'period_id_a0',
]
actions_columns = ['type_id',
                    'result_id',
                    'bodypart_id',
                    'team_id']
time_column = 'time_seconds_overall_a0'
seq_size = 100
label_encoders = {}
discrete_features_names = np.concatenate((features_discrete_columns, actions_columns))
real_features_names = np.concatenate((features_real_columns, ['vaep_value']))
discrete_features_value_counts = {}
features_column_index = {}
Xcols = np.concatenate((real_features_names, discrete_features_names))
Ycols = np.array(['substitution'])


neural_features = pd.read_hdf(neural_features_h5, 'all_neural_features')

for feature_name in discrete_features_names:
    lb_make = LabelEncoder()
    if not os.path.exists(neural_features_h5):
        neural_features[feature_name] = lb_make.fit_transform(neural_features[feature_name])
    else:
        neural_features[feature_name] = lb_make.fit(neural_features[feature_name])
    label_encoders[feature_name] = lb_make
    discrete_features_value_counts[feature_name] = len(lb_make.classes_)
    features_column_index[feature_name] = neural_features.columns.get_loc(feature_name)
for feature_name in real_features_names:
    features_column_index[feature_name] = neural_features.columns.get_loc(feature_name)
features_column_index[time_column] = neural_features.columns.get_loc(time_column)
del neural_features
action_dataset = ActionDataset(neural_features_h5, Xcols, Ycols, time_column, seq_size)

Loading data:   0%|          | 0/520 [00:00<?, ?it/s]

In [ ]:

training_length = int(0.7 * len(action_dataset))
validation_length = int((1 / np.sqrt(len(Xcols)) * training_length))
training_length = training_length - validation_length
testing_length = len(action_dataset) - training_length - validation_length

batch_size = 32

# hidden_weight = torch.randn(num_layers, batch_size, hidden_size)
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(action_dataset,
                                                                                [training_length, validation_length,
                                                                                    testing_length],
                                                                                generator=torch.Generator().manual_seed(
                                                                                    42))
# train_dataset = dataset
# test_dataset = dataset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(f'Size of training set : {len(train_dataset)}')
print(f'Size of validation set : {len(validation_dataset)}')
print(f'Size of testing set : {len(test_dataset)}')

Size of training set : 600424
Size of validation set : 172926
Size of testing set : 331437


In [ ]:
class ActionNeuralModel(nn.Module):
    def __init__(self, seq_size, embedding_size, time_embedding_size, real_features_names,
                 discrete_features_names,
                 discrete_features_value_counts, features_column_index, hidden_size, num_layers, device):
        super().__init__()
        self.seq_size = seq_size
        self.embedding_size = embedding_size
        self.time_embedding_size = time_embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
        self.embedding_layers = []

        self.real_features_names = real_features_names
        self.discrete_features_names = discrete_features_names
        self.discrete_features_value_counts = discrete_features_value_counts
        self.features_column_index = features_column_index
        self.embedding_layers = {}

        self.input_size = len(self.discrete_features_names) * self.embedding_size + len(
            self.real_features_names) + self.time_embedding_size
        for discrete_feature_name in discrete_features_names:
            self.embedding_layers[discrete_feature_name] = nn.Embedding(
                discrete_features_value_counts[discrete_feature_name], self.embedding_size).to(self.device)

        self.time_encoder = nn.Linear(1, out_features=self.hidden_size, bias=True).to(self.device)
        self.time_embedding_matrix = nn.Parameter(torch.zeros((self.hidden_size, self.time_embedding_size))).to(
            self.device)

        self.substitution_time_rnn = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True,
                                             dropout=0.2).to(self.device)
        self.linear_sub_time = nn.Linear(self.hidden_size, 1).to(self.device)
        self.activation_sub = nn.Sigmoid().to(self.device)

    def first_layer(self, x, time):
        split = torch.split(x, 1, dim=2)
        embedded_output_list = []
        for discrete_feature_name in self.discrete_features_names:
            #print(discrete_feature_name)
            #print(discrete_features_value_counts[discrete_feature_name])

            to_embed = split[features_column_index[discrete_feature_name]]
            embedding_layer = self.embedding_layers[discrete_feature_name]
            embedded_output = embedding_layer(to_embed.int()).to(device)
            embedded_output_list.append(torch.squeeze(embedded_output, dim=2))
        embedded_output = torch.stack(embedded_output_list, dim=3)
        embedded_output = torch.permute(embedded_output, (0, 1, 3, 2))
        embedded_output = torch.reshape(embedded_output, (embedded_output.shape[0], embedded_output.shape[1], -1))
        # embedded_output = torch.reshape(embedded_output, (
        #     embedded_output.shape[0], embedded_output.shape[1], embedded_output.shape[2] * embedded_output.shape[3]))
        real_features = []
        for real_feature_name in self.real_features_names:
            real_features.append(split[self.features_column_index[real_feature_name]])
        real_features = torch.squeeze(torch.stack(real_features, dim=2), dim=3)

        time_encoded = self.time_encoder(time)
        time_embedded = torch.einsum('bsv,vi->bsi', time_encoded, self.time_embedding_matrix)
        input = torch.cat([real_features, embedded_output, time_embedded], dim=2)
        input = input.float()
        return input

    def forward(self, x, time):
        hidden_weights = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(self.device)
        cell_weights = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(self.device)

        input = self.first_layer(x, time)
        submission_prob, _ = self.substitution_time_rnn(input, (hidden_weights.detach(), cell_weights.detach()))
        #print(submission_prob)
        #print(submission_prob[:, -1, :])
        #print(submission_prob[:, -1, :].shape)
        sub_raw = self.linear_sub_time(submission_prob[:, -1, :])
        #print(sub_raw)
        sub_prob = self.activation_sub(sub_raw)
        #print(sub_prob)
        return sub_prob


In [ ]:

embedding_size = 3
time_embedding_size = 6
hidden_size = 64
num_layers = 3
model = ActionNeuralModel(seq_size=seq_size,
                            embedding_size=embedding_size,
                            time_embedding_size=time_embedding_size,
                            real_features_names=real_features_names,
                            discrete_features_names=discrete_features_names,
                            discrete_features_value_counts=discrete_features_value_counts,
                            features_column_index=features_column_index,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            device=device).to(device)
loss_f = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:

eps = 1e-10


def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    training_step = 300
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in tqdm(enumerate(train_dataloader),
                        f'Training... Epoch : {epoch_index + 1} - Loss : {last_loss} - Dataset size : {len(train_dataloader)}'):
        # Every data instance is an input + label pair
        indices, game_id, action_indices, X, Time, Y = data
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        output = model(X, Time)
        #print(output)

        # Compute the loss and its gradients
        loss = loss_f(output + eps, Y)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000  # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            #print(f'GPU memory {torch.cuda.memory_allocated(device=device)}')
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
def save_ckp(state, is_best, checkpoint_dir, best_model_dir):
    if not os.path.exists(checkpoint_path):
        os.mkdir(checkpoint_path)
    f_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
    torch.save(state, f_path)


def load_ckp(checkpoint_fpath, model, optimizer):
    f_path = os.path.join(checkpoint_fpath, 'checkpoint.pt')
    checkpoint = torch.load(f_path)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']



In [ ]:

loss_value = 0

checkpoint_path = "./checkpoints/"
modelfolder = os.path.join(base_path, "action_model_folder")
writerfolder = os.path.join(modelfolder, 'runs')
os.makedirs(modelfolder, exist_ok=True)
os.makedirs(writerfolder, exist_ok=True)
num_epochs = 1
total_steps = 0
model_path = os.path.join(modelfolder, f'action-model-{num_epochs}-{batch_size}')

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
train = True
if train:
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter(os.path.join(writerfolder, 'action_model_{}'.format(timestamp)))

    EPOCHS = 5
    epoch_number = 0

    best_vloss = 1_000_000.
    
    for epoch_number in range(EPOCHS) :
        print('EPOCH {}:'.format(epoch_number))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(epoch_number, writer)

        # We don't need gradients on to do reporting
        torch.cuda.empty_cache()
        model.train(False)

        running_vloss = 0.0
        for i, vdata in tqdm(enumerate(validation_dataloader),
                                f'Evaluating... Epoch : {epoch_number + 1} - Loss : {running_vloss} - Dataset size : {len(validation_dataloader)}'):
            vindices, vgame_id, vaction_indices, vX, vTime, vY = vdata
            voutputs = model(vX, vTime)
            vloss = loss_f(voutputs, vY)
            running_vloss += vloss.item()

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                            {'Training': avg_loss, 'Validation': avg_vloss},
                            epoch_number + 1)
        writer.flush()

        checkpoint = {
            'epoch': epoch_number + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            torch.save(model.state_dict(), model_path)
        epoch_number += 1

    #os.remove(checkpoint_path)
else:
    model.load_state_dict(torch.load(model_path))

1
Tesla P100-PCIE-16GB
EPOCH 0:


Training... Epoch : 1 - Loss : 0.0 - Dataset size : 18763: 0it [00:00, ?it/s]

  batch 1000 loss: 0.275897915199399
  batch 2000 loss: 0.24853344745188952
  batch 3000 loss: 0.2527509559132159
  batch 4000 loss: 0.2482451119720936
  batch 5000 loss: 0.23940669737011194
  batch 6000 loss: 0.24272327338531613
  batch 7000 loss: 0.252559331856668
  batch 8000 loss: 0.24383446590229868
  batch 9000 loss: 0.24279148789867758
  batch 10000 loss: 0.24576329074054956
  batch 11000 loss: 0.24400876412540673
  batch 12000 loss: 0.24445802062749863
  batch 13000 loss: 0.24894931608811022
  batch 14000 loss: 0.24362761598080396
  batch 15000 loss: 0.24447663556039334
  batch 16000 loss: 0.24485770309716462
  batch 17000 loss: 0.24052810770645738
  batch 18000 loss: 0.24594871727004647


Evaluating... Epoch : 1 - Loss : 0.0 - Dataset size : 5403: 0it [00:00, ?it/s]

LOSS train 0.24594871727004647 valid 0.2433623858230568
EPOCH 1:


...

LOSS train 0.23769053326919676 valid 0.23437798508966431
EPOCH 4:


Training... Epoch : 5 - Loss : 0.0 - Dataset size : 18763: 0it [00:00, ?it/s]

  batch 1000 loss: 0.2423497897423804
  batch 2000 loss: 0.22702347708866
  batch 3000 loss: 0.22727596178650855
  batch 4000 loss: 0.23472790910676122
  batch 5000 loss: 0.2355998440682888
  batch 6000 loss: 0.23437628008425235
  batch 7000 loss: 0.23566500259563328
  batch 8000 loss: 0.23643790693208575
  batch 9000 loss: 0.22997200161963702
  batch 10000 loss: 0.23315535122156142
  batch 11000 loss: 0.22499111112952233
  batch 12000 loss: 0.2244999359510839
  batch 13000 loss: 0.23058983809500933
  batch 14000 loss: 0.23054795936495065
  batch 15000 loss: 0.22785012459754944
  batch 16000 loss: 0.227551565349102
  batch 17000 loss: 0.22790524324029685
  batch 18000 loss: 0.23039833079650998


Evaluating... Epoch : 5 - Loss : 0.0 - Dataset size : 5403: 0it [00:00, ?it/s]

LOSS train 0.23039833079650998 valid 0.22701366579181814


In [ ]:

batch_size = 128
dataloader = DataLoader(action_dataset, batch_size=batch_size, shuffle=False)
with torch.no_grad():
    model.eval()
    Y_test = []
    predictions = []
    indices_list = []
    game_id_list = []
    action_indices_list = []
    for indices, game_id, action_indices, X, Time, Y in tqdm(dataloader,
                                                    'Predicting substitution for action dataset...'):
        output = model(X, Time)
        output = torch.reshape(output, (X.shape[0], 1)).cpu()
        output = torch.squeeze(output)
        predictions.append(output.numpy())
        Y = torch.squeeze(Y).cpu()
        Y_test.append(Y.numpy())
        indices_list.append(indices.numpy())
        game_id_list.append(game_id.numpy())
        action_indices_list.append(action_indices.numpy())

predictions_store = predictions
Y_test_store = Y_test
indices_list_store = indices_list
action_list_store = action_indices_list
game_id_store = game_id_list

Predicting substitution for action dataset...:   0%|          | 0/8632 [00:00<?, ?it/s]

In [ ]:
def evaluate(y, y_hat):
    print(f'Number of datapoints : {len(y)}')
    p = sum(y) / len(y)
    base = [p] * len(y)
    brier = brier_score_loss(y, y_hat)
    print(f"  Brier score: %.5f (BSS %.5f)" % (brier, 1 - (brier / brier_score_loss(y, base))))
    ll = log_loss(y, y_hat)
    print(f"  log loss score: %.5f (%.5f)" % (ll, ll / log_loss(y, base)))
    print(f"  ROC AUC: %.5f" % roc_auc_score(y, y_hat))

In [ ]:
print(predictions_store[0].shape)
print(Y_test_store[0].shape)
print(indices_list_store[0].shape)
print(action_list_store[0].shape)

predictions = np.ndarray((len(predictions_store), batch_size))
Y_test = np.ndarray((len(Y_test_store), batch_size))
indices_list = np.ndarray((len(indices_list_store), batch_size))
actions_list = np.ndarray((len(action_list_store), batch_size))
game_id_list = np.ndarray((len(game_id_store), batch_size))

for i in tqdm(range(len(predictions_store))) :
    predictions[i][0:len(predictions_store[i])] = predictions_store[i]
    Y_test[i][0:len(Y_test_store[i])] = Y_test_store[i]
    indices_list[i][0:len(indices_list_store[i])] = indices_list_store[i]
    actions_list[i][0:len(action_list_store[i])] = action_list_store[i][:, -1]
    game_id_list[i][0:len(game_id_store[i])] = game_id_store[i]

predictions = predictions.flatten()
Y_test = Y_test.flatten()
indices_list = indices_list.flatten()
actions_list = actions_list.flatten()
game_id_list = game_id_list.flatten()

print(predictions.shape)
print(Y_test.shape)
print(indices_list.shape)
print(actions_list.shape)
print(game_id_list.shape)
print(predictions[0])
print(Y_test[0])
print(indices_list[0])
print(actions_list[0])
print(game_id_list[0])

np_data = np.array([indices_list, game_id_list, actions_list, Y_test, predictions])
np_data = np.swapaxes(np_data, axis1=0, axis2=1)
print(np_data)
data = pd.DataFrame(np_data, columns=['indice','game_id', 'action_id', 'true', 'pred'])
print(data.shape)

(128,)
(128,)
(128,)
(128, 100)


  0%|          | 0/8632 [00:00<?, ?it/s]

(1104896,)
(1104896,)
(1104896,)
(1104896,)
(1104896,)
0.0036599228624254465
0.0
0.0
99.0
70270.0
[[0.00000000e+000 7.02700000e+004 9.90000000e+001 0.00000000e+000
  3.65992286e-003]
 [1.00000000e+000 7.02700000e+004 1.00000000e+002 0.00000000e+000
  3.68330907e-003]
 [2.00000000e+000 7.02700000e+004 1.01000000e+002 0.00000000e+000
  3.71775590e-003]
 ...
 [6.92902955e-310 0.00000000e+000 0.00000000e+000 2.38139641e-321
  0.00000000e+000]
 [6.92902955e-310 0.00000000e+000 0.00000000e+000 2.38633707e-321
  4.94065646e-324]
 [6.92902955e-310 0.00000000e+000 0.00000000e+000 2.39127773e-321
  0.00000000e+000]]
(1104896, 5)


In [ ]:
for game in tqdm(action_dataset.games.itertuples()) :
    game_id = game.game_id
    game_data = data[data['game_id'] == game_id]
    game_data = game_data.sort_values('action_id', ascending=True).reset_index()
    game_data = game_data[(game_data['true'] == 0) | (game_data['true'] == 1)]
    game_data['true'].to_hdf(neural_predictions_h5, f'target/game_{game_id}')
    game_data['pred'].to_hdf(neural_predictions_h5, f'predictions/game_{game_id}')
   

0it [00:00, ?it/s]

In [ ]:
data = data[(data['true'] == 0) | (data['true'] == 1)]
y = data['pred']
y_hat = data['true']

#y = y[(y_hat == 0) | (y_hat == 1)]
#y_hat = y_hat[(y_hat == 0) | (y_hat == 1)]

print(y)
print(y_hat)

y = y.to_numpy()
y_hat = y_hat.to_numpy()
evaluate(y_hat, y)

0          0.003660
1          0.003683
2          0.003718
3          0.004484
4          0.009110
             ...   
1104782    0.131923
1104783    0.135873
1104784    0.117617
1104785    0.127613
1104786    0.127061
Name: pred, Length: 1104787, dtype: float64
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1104782    0.0
1104783    0.0
1104784    0.0
1104785    0.0
1104786    0.0
Name: true, Length: 1104787, dtype: float64
Number of datapoints : 1104787
  Brier score: 0.06476 (BSS 0.12036)
  log loss score: 0.22578 (0.80976)
  ROC AUC: 0.81481


In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

for game in tqdm(action_dataset.games.itertuples()) :
    fig = go.Figure()
    print(game)
    game_id = game.game_id
    game_features = pd.read_hdf(spadl_h5, f'actions/game_{game.game_id}')
    print(game_features)
    game_data = data[data['game_id'] == game_id]
    print(game_data)
    merged = pd.merge(game_features, game_data, left_index=True, right_index=True)
    barca_actions = merged[merged['team_id'] == 217]
    print(merged)
    x = barca_actions['indice']
    y = barca_actions['pred']
    subs = barca_actions['true']

    fig.add_trace(
            go.Scatter(
                name=f"test",
                mode="lines",
                x=x,
                y=y,
            )
        )
    fig.add_trace(
            go.Scatter(
                name=f"subs",
                mode="lines",
                x=x,
                y=subs,
            )
        )
    #fig.write_html(os.path.join(base_path, f'plots/average-probabilities-6-features-batch-{i}.html'))
    fig.show()
    print(gdf)

0it [00:00, ?it/s]

Pandas(Index=275, index=1, game_id=70270, season_id=24, competition_id=11, competition_stage='Regular Season', game_day=5, game_date=Timestamp('2012-09-22 22:00:00'), home_team_id=217, away_team_id=1049, home_score=2, away_score=0, venue='Camp Nou', referee=nan)
      game_id                     original_event_id  period_id  time_seconds  \
0       70270  fc649eba-d85b-4676-829f-e3fac85456be          1           0.0   
1       70270  f8586b6f-cc6b-41d0-8704-67734e3627fa          1           0.0   
2       70270  e81cb97a-5de6-4baa-bc01-475a815521de          1           1.0   
3       70270  6a79f73c-0fdb-47f2-adc5-ea993c2e074e          1           2.0   
4       70270  44085b17-5fd6-4f0d-befb-421fca6fe3a3          1           3.0   
...       ...                                   ...        ...           ...   
2412    70270  f0f33e02-b382-4efd-884f-ebdb1e84fbd4          2        2873.0   
2413    70270  228b5bc9-941a-4c97-b9c9-937b5b934771          2        2873.0   
2414    70270  bf

NameError: ignored

In [ ]:
predictions = np.array(predictions_store)
print(predictions.shape)
predictions = np.expand_dims(predictions, axis=-1)
predictions = pd.DataFrame(predictions, columns=['substitution'])
print(predictions.loc[1]['substitution'].shape)
Y_test = np.array(Y_test_store).flatten()
Y_test = pd.DataFrame(Y_test, columns=['substitution'])
indices_list = np.array(indices_list_store)
action_indices_list = np.array(action_list_store)
indices_list = np.expand_dims(indices_list, axis=-1)
action_indices_list = np.expand_dims(action_indices_list, axis=-1)
print(type(predictions))

action_indices_list = np.concatenate([action_indices_list, indices_list], axis=-1)
indices_list = pd.DataFrame(indices_list, columns=['action_id'])
action_indices_list = pd.DataFrame(action_indices_list, columns=['action_id', 'game_id'])
predictions[['action_id', 'game_id']] = action_indices_list[['action_id', 'game_id']]
Y_test[['action_id', 'game_id']] = action_indices_list[['action_id', 'game_id']]
predictions = predictions.drop_duplicates()
Y_test = Y_test.drop_duplicates()
predictions.to_hdf(neural_predictions_h5, 'predictions')
Y_test.to_hdf(neural_predictions_h5, 'target')
predictions = pd.read_hdf(neural_predictions_h5, 'predictions')
for game in tqdm(games_df):
    game_substitutions = Y_test[Y_test['game_id'] == game]
    game_predictions = predictions[predictions['game_id'] == game]
    game_substitutions = game_substitutions.sort_values('action_id', ascending=True).reset_index()
    game_predictions = game_predictions.sort_values('action_id', ascending=True).reset_index()
    game_substitutions.to_hdf(neural_predictions_h5, f'target/game_{game}')
    game_predictions.to_hdf(neural_predictions_h5, f'predictions/game_{game}')

y = predictions.drop_duplicates()
y_hat = Y_test.drop_duplicates()

y = y['substitution'].to_numpy()
y_hat = y_hat['substitution'].to_numpy()
evaluate(y_hat, y)